# Topic modeling
En este notebook se va a demostrar el uso de distintos modelos de extracción de temáticas (*topic modeling*) en un conjunto de textos de ejemplo sencillo.

In [ ]:
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

nlp=spacy.load('en_core_web_md')

### Creación del corpus
Creamos un pequeño Corpus de ejemplo formado por 8 frases cortas. Definimos una sencilla función de normalización y aplicamos esta normalización a todo el corpus.

In [ ]:
def normalize_document(doc):
    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios/stop words y cogemos el lema
    lemmas = [t.lemma_ for t in tokens if not t.is_punct and not t.is_space and not t.is_stop]
    doc = ' '.join(lemmas)
    return doc

def normalize_corpus(corpus):
    """Normaliza un corpus de documentos aplicando al función de normalización
    normalize_document() a cada documento de la lista pasada como argumento"""   
    return [normalize_document(text) for text in corpus]

toy_corpus = [
"The fox jumps over the dog",
"the fox is very clever and quick",
"The dog is slow and lazy",
"The cat is smarter than the fox and the dog",
"Python is an excellent programming language",
"Java and Ruby are other programming languages",
"Python and Java are very popular programming languages",
"Python programs are smaller than Java programs"]

norm_corpus = normalize_corpus(toy_corpus)
norm_corpus

## Topic modeling usando Scikit-learn
La librería `scikit-learn` implementa los modelos *Latent Semantic Analysis* (LSA) y *Latent Dirichlet Allocation* (LDA).  
Partimos de un modelo TF-IDF para el modelado LSA y de un modelo BoW para el modelado LDA

### Modelo LSA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# usamos características tf-idf para LSA.
tfidf_vectorizer = TfidfVectorizer(min_df=2)
tfidf = tfidf_vectorizer.fit_transform(norm_corpus)

In [ ]:
tfidf_vectorizer.get_feature_names_out()

Definimos una función de ayuda para mostrar los resultados (términos asociados a cada tema)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    """Función auxiliar para mostrar los términos más importantes
    de cada topic"""
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

Calculamos los modelos para nuestro corpus (método `fit`) y vemos cuáles son los 3 términos con más peso para cada *topic*. Cada modelo asigna un grado de pertenencia en cada tema a cada término del vocabulario de la matriz tfidf o bow utilizada como entrada.

In [ ]:
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

# Ajustamos el modelo LSA
lsa = TruncatedSVD(n_components=2).fit(tfidf)

print("\nTopics en modelo LSA:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lsa, tfidf_feature_names, 3)

El método `fit` aprende la matriz de `topics` x `términos` para el corpus dado

In [ ]:
lsa.components_.shape

In [ ]:
pd.DataFrame(np.round(lsa.components_, 4), columns=tfidf_feature_names)

In [ ]:
tfidf.todense().shape

Podemos ver el porcentaje de pertenencia a cada *topic* de cada una de los documentos asignados por el modelo con el método `transform`:

In [ ]:
lsa.transform(tfidf)

In [ ]:
np.round(lsa.transform(tfidf), 4)

Cada fila corresponde a un documento del Corpus, y cada columna el grado de pertenencia a ese tema del documento.  
El modelo ha separado correctamente el corpus en las dos temáticas principales:

In [ ]:
np.argmax(lsa.transform(tfidf), axis=1)

### Modelo LDA

In [ ]:
# usamos características BoW para LDA.
tf_vectorizer = CountVectorizer(min_df=2)
tf = tf_vectorizer.fit_transform(norm_corpus)

In [ ]:
tf

In [ ]:
# Ajustamos el modelo LDA
lda = LatentDirichletAllocation(n_components=2, max_iter=5,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=0).fit(tf)

print("\nTopics en modelo LDA:")
tf_feature_names = tf_vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, 3)

El atributo `components_` contiene los parámetros de la distribución de términos en *topics*.

In [ ]:
lda.components_.shape

In [ ]:
pd.DataFrame(lda.components_, columns=tfidf_feature_names)

Normalizando esta matriz muestra la distribución de términos dentro de cada *topic*

In [ ]:
distribucion = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
pd.DataFrame(distribucion, columns=tfidf_feature_names)

Como en el caso del LSA, podemos ver el grado de pertenencia de cada documento a cada *topic*

In [ ]:
lda.transform(tf)

In [ ]:
lda.transform(tf).shape

In [ ]:
np.sum(lda.transform(tf), axis=1)

In [ ]:
np.argmax(lda.transform(tf), axis=1)

## Topic modeling usando librería Gensim
La librería `gensim` implementa los siguientes modelos:  
* [Latent Semantic Indexing, LSI (or sometimes LSA)](https://en.wikipedia.org/wiki/Latent_semantic_indexing) transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality.  
* [Latent Dirichlet Allocation, LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).  
* [Hierarchical Dirichlet Process, HDP](http://jmlr.csail.mit.edu/proceedings/papers/v15/wang11a/wang11a.pdf) is a non-parametric bayesian method (note the missing number of requested topics.

La entrada a los modelos de `gensim`
 debe ser una lista de tokens y no un texto por cada documento del corpus, por lo que hay que cambiar la función de normalización.

In [ ]:
def normalize_tokenize_document(doc):
    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios y cogemos el lema
    lemmas = [t.lemma_.lower() for t in tokens if not t.is_punct and not t.is_space and not t.is_stop]
    return lemmas

def normalize_tokenize_corpus(corpus):
    """Normaliza un corpus de documentos aplicando al función de normalización
    normalize_tokenize_document() a cada documento de la lista pasada como argumento"""   
    return [normalize_tokenize_document(text) for text in corpus]
        
norm_tokenized_corpus = normalize_tokenize_corpus(toy_corpus)
norm_tokenized_corpus

In [ ]:
norm_corpus

Al igual que en los modelos de la librería `scikit-learn`, primero generamos matrices de características BoW y TF-IDF como paso previo a aplicar los modelos de topic-modeling.  
En `gensim` estas matrices se calculan de manera diferente a `scikit-learn`

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, TfidfModel

#diccionario de términos únicos del corpus
dictionary = Dictionary(norm_tokenized_corpus)
#creamos matriz BoW
corpus_bow = [dictionary.doc2bow(text)
                 for text in norm_tokenized_corpus]
#creamos matriz TF-IDF del corpus a partir de BoW
tfidf = TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [ ]:
corpus_bow[0]

In [ ]:
corpus_tfidf[0]

In [ ]:
corpus_tfidf

Cada tupla indica la frecuencia del ítem *i* en el documento, donde *i* es el índice de las palabras en el vocabulario del diccionario de Gensim.

In [ ]:
[(i, k) for i,k in dictionary.items()]

### Latent Semantic Indexing
Los modelos de *topic modeling* de `gensim` asignan un peso de pertenencia de cada término del diccionario bow/tfidf a cada tema:

In [ ]:
lsi = LsiModel(corpus_tfidf, 
                      id2word=dictionary,
                      num_topics=2)

for index, topic in lsi.print_topics(2):
    print(f'Topic #{str(index)}\n{topic}\n')

In [ ]:
lsi.get_topics().shape

La matriz LSI generada es un objeto específico de Gensim que funciona como un *iterable*. Contiene el grado de pertenencia al *topic* o *topics* más representativos del documento.

In [ ]:
topics_lsi = lsi[corpus_tfidf]
topics_lsi

In [ ]:
topics_lsi[0]

In [ ]:
for t in topics_lsi:
    print(t)

El objeto *TransformedCorpus* sólo muestra los componentes distintos de cero de la matriz LSI del corpus:

In [ ]:
import gensim
gensim.matutils.corpus2dense(lsi[corpus_tfidf], len(lsi.projection.s)).T 

### Latent Dirichlet Allocation

In [ ]:
lda = LdaModel(corpus_bow, 
                      id2word=dictionary,
                      iterations=1000,
                      num_topics=2)
for index, topic in lda.print_topics(2):
    print('Topic #{}\n{}\n'.format(str(index), topic))

In [ ]:
topics_lda = lda[corpus_bow]
topics_lda

In [ ]:
topics_lda[0]

In [ ]:
for t in topics_lda:
    print(t)

In [ ]:
lda.get_topics().shape

### Hierarchical Dirichlet Process

In [ ]:
#no hay que especificar un núm. de topics
hdp = HdpModel(corpus_bow, 
                      id2word=dictionary)
for index, topic in hdp.print_topics(2):
    print('Topic #{}\n{}\n'.format(str(index), topic))

In [ ]:
for index, topic in hdp.print_topics(8):
    print('Topic #{}\n{}\n'.format(str(index), topic))

In [ ]:
topics_hdp = hdp[corpus_bow]
topics_hdp

In [ ]:
for t in topics_hdp:
    print(t)

### Estimación de temática principal
Podemos calcular la pertenencia de cada documento a una temática a partir de su modelo calculado:  
El modelo LSI sólo devuelve el grado de pertencia de cada documento a los *topics* más relevantes a ese documento.

In [ ]:
corpus_lsi = lsi[corpus_tfidf]
for i, doc in enumerate(corpus_lsi):
     print(doc, toy_corpus[i])

Sin embargo LDA y HDP calculan la pertenencia a cada tema y devuelven una lista de tuplas por cada tema (nº de tema, grado de pertenencia).

In [ ]:
corpus_lda = lda[corpus_bow]
for i, doc in enumerate(corpus_lda):
     print(doc, toy_corpus[i])

En el modelo LDA el peso de cada palabra en cada *topic* es una probabilidad

In [ ]:
lda.get_topics()

In [ ]:
lda.get_topics().shape

In [ ]:
np.sum(lda.get_topics(), axis=1)

Y las pertenencias de un documento a cada *topic* también son un valor de probabilidad

In [ ]:
gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T

In [ ]:
np.sum(gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T, axis=1)

In [ ]:
np.argmax(gensim.matutils.corpus2dense(lda[corpus_bow], lda.num_topics).T, axis=1)

Con el modelo HDP no se especifica un número de temas sino que se definen automáticamente (con importancia decreciente)

In [ ]:
# Solución
corpus_hdp = hdp[corpus_bow]
for i, doc in enumerate(corpus_hdp):
     print(doc, toy_corpus[i])

Podemos obtener los términos relevantes para cada tema y su importancia con el método `show_topics` del modelo:

In [ ]:
lsitopics = [[(word,prob) for word, prob in topic] for topicid, topic in lsi.show_topics(formatted=False)]

hdptopics = [[(word,prob) for word, prob in topic] for topicid, topic in hdp.show_topics(formatted=False)]

ldatopics = [[(word,prob) for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]

In [ ]:
ldatopics

In [ ]:
lsitopics

### Topic Coherence
La librería `gensim` proporciona una funcionalidad para identificar qué modelo de *topic modeling* se adapta mejor al corpus. La función `CoherenceModel` calcula una puntuación sobre la coherencia del modelo, que podemos usar para compararlos. Esta función utiliza las palabras que definen cada tópico en los modelos.

In [ ]:
lsitopics = [[word for word, prob in topic] for topicid, topic in lsi.show_topics(formatted=False)]

hdptopics = [[word for word, prob in topic] for topicid, topic in hdp.show_topics(formatted=False)]

ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]

lsi_coherence = CoherenceModel(topics=lsitopics[:10], texts=norm_tokenized_corpus,
                               dictionary=dictionary, window_size=10).get_coherence()

hdp_coherence = CoherenceModel(topics=hdptopics[:10], texts=norm_tokenized_corpus, 
                               dictionary=dictionary, window_size=10).get_coherence()

lda_coherence = CoherenceModel(topics=ldatopics, texts=norm_tokenized_corpus,
                               dictionary=dictionary, window_size=10).get_coherence()

In [ ]:
lsitopics

In [ ]:
lsi_coherence

In [ ]:
def evaluate_bar_graph(coherences, indices):
    """
    Función para dibujar una gráfica de barras con:
    
    coherences: lista de los valores de coherencia
    indices: textos para etiquetar las barras.
    Ambos parámetros deben tener la misma longitud
    """
    assert len(coherences) == len(indices)
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Modelos')
    plt.ylabel('Valor Coherencia')

In [ ]:
evaluate_bar_graph([lsi_coherence, hdp_coherence, lda_coherence],
                   ['LSI','HDP', 'LDA'])